## 불용어를 빼내자

In [1]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# NLP
import nltk
from soynlp.normalizer import *
from hanspell import spell_checker
from konlpy.tag import Okt

# ML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

# 한글화
import koreanize_matplotlib 

In [2]:
# 파일 불러오기

review_df = pd.read_csv('./review_all_hanspell_stop3.csv', encoding='utf-8', index_col=0) # df_review는 전체년도 리뷰 크롤링 파일
review_df.drop(['tokenized_review', 'tokenized_removed_review','reviews_for_vectorize'], axis=1, inplace=True)
review_df.head()

,date,review,rating
0,2023-09-10,상세 설명에 들어가지 않아도 검색 결과에서 중량과 가격이 바로 확인되면 더 편리하겠습니다,5
1,2023-09-09,필터 기능 잘 되어있어 편하게 쇼핑함,5
2,2023-09-09,배송이 빨라요 그리고 할인이 많아요,5
3,2023-09-09,50퍼 쿠폰 만원 빵빵하게 주는 곳 처음 봄 주문하면 6시간 만에 옴 이 앱 뭐지,5
4,2023-09-09,빠른 배송,5


In [3]:
# 일단 토큰화 - 명사
from konlpy.tag import Okt

tokenize = Okt()

review_df['review_tokens'] = [ tokenize.nouns(re) for re in review_df['review'] ]

In [4]:
# 불용어 제거 : RankNL encoding='utf-8'
with open('stopwords.txt', 'r') as f:
    stopwords = f.readlines()
    stopwords = [word.strip() for word in stopwords]
#print('RankNL 불용어 개수 : ' + str(len(stopwords)))

# 불용어 제거 : 수작업
stopwords_to_add = ['편','재','한눈','샛별','폐','수고','부탁','안정','가능','식','경기도','강원도','전과','미만','경우','항상','러','일해','지난','세','밀','케이크','매우','만큼','오늘','낼','전','광주','출석','개보','최대','물이','생기','통해','귀사','무궁','발전','기원','번창','뭘','풀','절','짐','바라지','사려','강','기껏','나중','알','비위','구분','전달','주어','나이스','감사','제발','개밥','간식','최소한','구경','기대','갈수록','일지','끝','확장','모든','사랑','제주도','짜증','갑자기','해결','실행','자동','페이','별명','중','사유','해당','표시','당황','빨','물','뒤','가기','사용','사람','파이팅','가성','비','태깡','검색','설정','아예','게','쿠팡','문제','마지막','적지','치면','결과','돌이','필수','꼭','살','요즘','추가','제안','기분','제외','질','밤샘','상','일정','선','임','최고','진짜','진작','이용','수가','계산','쓱','이전','조율','예상','오오오오','욥','제대로','말','요청','부서','담당','개선','가입','낮','볼','늘','고요','원','쇼핑','애용','주니','마음','드네','정말','아주','샴푸','남편','물건','걸','자주','하라','앞','뷰티','규','사고','다리','컬리','리','퍼퓸','플루','켄트','칫솔','궁','향','마켓','짱짱맨','확인','빵빵','처음','봄','앱','함','곳', '옴', '뭐', '더', '이', '에', '도', '가', '을', '안', '하고', '것', '들', '를', '로', '수', '할', '으로', '거', '은', '한', '다', '만', '는', '적', '후', '하기', '요', '해', '에서', '있어요', '입니다', '여러', '개', '번', '되는', '보니', '하겟슴', '했고요', '제', '라', '마', '해요', '지', '있음', '과', '의', '인데', '하기에']
stopwords.extend(stopwords_to_add)
print('수작업 불용어 개수 : ' + str(len(stopwords_to_add)))
print('총 불용어 개수 : ' + str(len(stopwords)))

수작업 불용어 개수 : 222
총 불용어 개수 : 1024


In [5]:
def remove_stopwords(tokens):
    words = list(set([word for word in tokens if word not in stopwords]))
    return words
review_df['review_tokens_removeST'] = review_df['review_tokens'].apply(remove_stopwords)

In [6]:
review_df['review_tokens_removeST'][130:140]

131                                              []
132    [상품, 대량, 인지, 물량, 상의, 가격, 쿠폰, 품절, 할인, 대가, 주문]
133                                              []
134                                            [상품]
135                                              []
136                                              []
137                                            [배송]
138                                       [무료, 배송비]
139                                            [배송]
140                [자체, 회사, 실제, 점수, 역대, 다시, 혈압, 수준]
Name: review_tokens_removeST, dtype: object

In [7]:
## 불용어 텍스트 파일 업데이트

In [8]:
file_name = './stopwords.txt'
stopwords = list(set(stopwords))
with open(file_name, 'w+') as file: # w+ 안에 내용을 싹 지우고 다시 
    file.write('\n'.join(stopwords))  # '\n' 대신 ', '를 사용하면 줄바꿈이 아닌 ', '를 기준으로 문자열 구분함

## 결칙치 제거

In [9]:
review_df['review_tokens_removeST'] = [None if len(tokens)==0 else tokens for tokens in review_df['review_tokens_removeST']]
review_df.dropna(axis=0,inplace=True)
review_df.reset_index(drop=True,inplace=True)
review_df.head(10)

,date,review,rating,review_tokens,review_tokens_removeST
0,2023-09-10,상세 설명에 들어가지 않아도 검색 결과에서 중량과 가격이 바로 확인되면 더 편리하겠습니다,5,"[상세, 설명, 검색, 결과, 중량, 가격, 바로, 확인, 더]","[가격, 설명, 상세, 중량]"
1,2023-09-09,필터 기능 잘 되어있어 편하게 쇼핑함,5,"[필터, 기능, 쇼핑, 함]","[기능, 필터]"
2,2023-09-09,배송이 빨라요 그리고 할인이 많아요,5,"[배송, 할인]","[배송, 할인]"
3,2023-09-09,50퍼 쿠폰 만원 빵빵하게 주는 곳 처음 봄 주문하면 6시간 만에 옴 이 앱 뭐지,5,"[쿠폰, 만원, 빵빵, 곳, 처음, 봄, 주문, 옴, 이, 앱, 뭐]","[만원, 쿠폰, 주문]"
4,2023-09-09,빠른 배송,5,[배송],[배송]
5,2023-09-09,컬리 너무 좋다 리 뷰티 쿠폰 줘가지 규 사고팠던 퍼퓸이랑 플루랑 켄트 칫솔도 사궁...,5,"[컬리, 리, 뷰티, 쿠폰, 규, 사고, 퍼퓸, 플루, 켄트, 칫솔, 궁, 향, 기...",[쿠폰]
6,2023-09-08,마켓 컬리 대박이에요,5,"[마켓, 컬리, 대박]",[대박]
7,2023-09-08,배송이 완전 빠르고 제품 포장이 잘 되어와서 만족하고 있습니다,5,"[배송, 완전, 제품, 포장]","[제품, 포장, 배송, 완전]"
8,2023-09-08,아니 컬리 팜 초대해도 안 열리던데,1,"[컬리, 초대, 안]",[초대]
9,2023-09-07,샴푸를 저렴한 가격에 구매하게 되어 너무 남편이 물건 사는 걸 좋아하지 않는데 컬...,5,"[샴푸, 가격, 구매, 남편, 물건, 걸, 컬리, 쿠폰, 때, 자주, 구매, 하라,...","[가격, 구매, 상품, 쿠폰]"


In [10]:
# 벡터화에 필요한 작업 (띄어쓰기 붙여주기)

contents_for_vectorize = []

for content in review_df['review_tokens_removeST']: 
    sentence = ""
    for word in content:
        sentence = sentence + ' ' + word
        
    contents_for_vectorize.append(sentence)

In [11]:
review_df['review_tokens_removeST_vec'] = contents_for_vectorize
review_df.sample(5)

,date,review,rating,review_tokens,review_tokens_removeST,review_tokens_removeST_vec
3849,2021-09-02,집에 그냥 있을 땐 새벽 배송이 좋았는데 출근하다 보면 냉동제품 냉장제품이 녹을까 ...,2,"[집, 그냥, 땐, 새벽, 배송, 출근, 냉동, 제품, 냉장, 제품, 선택, 수, ...","[완료, 선택, 배송, 냉동, 제품, 냉장, 새벽, 집, 땐, 니요, 그냥, 지역,...",완료 선택 배송 냉동 제품 냉장 새벽 집 땐 니요 그냥 지역 출근 충청
8431,2020-03-13,배송도 상품도 모두 만족스러워요 앞으로 자주 애용할 것 같아요,5,"[배송, 상품, 모두, 앞, 자주, 애용, 것]","[상품, 배송]",상품 배송
1746,2022-06-16,배송 빠르고 상품 깔끔하고 좋은데 다른 곳 보다 비싼 느낌 돈 여유만 있다면 마켓...,3,"[배송, 상품, 다른, 곳, 느낌, 돈, 여유, 마켓, 컬리, 듯, 저, 가성, 비...","[느낌, 상품, 배송, 듯, 돈, 여유]",느낌 상품 배송 듯 돈 여유
2518,2022-03-02,다 좋은데 렉이 자주 있어요 상품 들어갈 때나 결제할 때,4,"[렉, 자주, 상품, 때, 결제, 때]","[결제, 렉, 상품]",결제 렉 상품
103,2023-07-25,신선하고 신속하고 안전한 배송에 신뢰가 갑니다,5,"[배송, 신뢰]","[배송, 신뢰]",배송 신뢰


In [12]:
# 라벨링
review_df['label'] = [1 if rate >=4 else 0 for rate in review_df['rating']]
review_df.head()

,date,review,rating,review_tokens,review_tokens_removeST,review_tokens_removeST_vec,label
0,2023-09-10,상세 설명에 들어가지 않아도 검색 결과에서 중량과 가격이 바로 확인되면 더 편리하겠습니다,5,"[상세, 설명, 검색, 결과, 중량, 가격, 바로, 확인, 더]","[가격, 설명, 상세, 중량]",가격 설명 상세 중량,1
1,2023-09-09,필터 기능 잘 되어있어 편하게 쇼핑함,5,"[필터, 기능, 쇼핑, 함]","[기능, 필터]",기능 필터,1
2,2023-09-09,배송이 빨라요 그리고 할인이 많아요,5,"[배송, 할인]","[배송, 할인]",배송 할인,1
3,2023-09-09,50퍼 쿠폰 만원 빵빵하게 주는 곳 처음 봄 주문하면 6시간 만에 옴 이 앱 뭐지,5,"[쿠폰, 만원, 빵빵, 곳, 처음, 봄, 주문, 옴, 이, 앱, 뭐]","[만원, 쿠폰, 주문]",만원 쿠폰 주문,1
4,2023-09-09,빠른 배송,5,[배송],[배송],배송,1


In [13]:
review_df

,date,review,rating,review_tokens,review_tokens_removeST,review_tokens_removeST_vec,label
0,2023-09-10,상세 설명에 들어가지 않아도 검색 결과에서 중량과 가격이 바로 확인되면 더 편리하겠습니다,5,"[상세, 설명, 검색, 결과, 중량, 가격, 바로, 확인, 더]","[가격, 설명, 상세, 중량]",가격 설명 상세 중량,1
1,2023-09-09,필터 기능 잘 되어있어 편하게 쇼핑함,5,"[필터, 기능, 쇼핑, 함]","[기능, 필터]",기능 필터,1
2,2023-09-09,배송이 빨라요 그리고 할인이 많아요,5,"[배송, 할인]","[배송, 할인]",배송 할인,1
3,2023-09-09,50퍼 쿠폰 만원 빵빵하게 주는 곳 처음 봄 주문하면 6시간 만에 옴 이 앱 뭐지,5,"[쿠폰, 만원, 빵빵, 곳, 처음, 봄, 주문, 옴, 이, 앱, 뭐]","[만원, 쿠폰, 주문]",만원 쿠폰 주문,1
4,2023-09-09,빠른 배송,5,[배송],[배송],배송,1
...,...,...,...,...,...,...,...
9921,2016-03-02,모바일로 쉽게 살 수 있어서,5,"[모바일, 살, 수]",[모바일],모바일,1
9922,2016-03-02,아이를 키우며 좋은 식자재에 대한 갈증이 있는데 이렇게 건강하고 신선한 먹거리를 ...,5,"[아이, 식, 자재, 대한, 갈증, 먹거리, 접, 수, 컴퓨터, 시간, 엄마, 앱,...","[자재, 엄마, 처럼, 컴퓨터, 식탁, 지금, 접, 먹거리, 대한, 갈증]",자재 엄마 처럼 컴퓨터 식탁 지금 접 먹거리 대한 갈증,1
9923,2016-03-02,아는 동생 소개로 알게 된 마켓 컬리 늦게 안 게 아쉬울 정도 제품퀄리티완젼좋고포장...,5,"[동생, 소개, 알, 마켓, 컬리, 안, 게, 정도, 제품, 퀄리티, 완젼, 포장,...","[완젼, 눈, 분, 사도, 가족, 동생, 배송, 안보, 제품, 인, 인제, 양도, ...",완젼 눈 분 사도 가족 동생 배송 안보 제품 인 인제 양도 정도 퀄리티 장 강추 ...,1
9924,2016-03-02,샛별 배송 정말 훌륭합니다 컬리라면 무조건 믿고 주문,5,"[샛별, 배송, 정말, 컬리, 라면, 무조건, 주문]","[무조건, 라면, 배송, 주문]",무조건 라면 배송 주문,1


In [14]:
review_df.to_csv("[JHS]preprocessed_data.csv", encoding='utf-8')